In [1]:
import pandas as pd
import sqlite3

## 1

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## 2

In [ ]:
pd.io.sql.read_sql(
    """
    SELECT * FROM test
    WHERE uid = "user_1"
    ORDER BY first_commit_ts;
    """,
    con=conn
)

,index,uid,labname,first_commit_ts,first_view_ts
0,20,user_18,laba04,2020-04-26 19:48:11.822365,2020-04-26 22:49:29.243278
1,21,user_18,laba04s,2020-04-26 20:03:56.935458,2020-04-26 22:49:29.243278
2,33,user_18,laba05,2020-05-03 13:01:34.848756,2020-04-26 22:49:29.243278
3,38,user_18,project1,2020-05-11 10:10:04.166881,2020-04-26 22:49:29.243278


In [15]:
pd.io.sql.read_sql(
    """
    SELECT t.uid, t.labname, t.first_commit_ts, t.first_view_ts, DATETIME(d.deadlines, 'unixepoch') AS deadlines
    FROM test t LEFT JOIN deadlines d ON t.labname = d.labs
    limit 3;
    """,
    con=conn
)

,uid,labname,first_commit_ts,first_view_ts,deadlines
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899,2020-05-16 23:59:59
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035,2020-04-26 23:59:59
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035,2020-04-26 23:59:59


In [19]:
test_results = pd.io.sql.read_sql(
    """
    SELECT 
        CASE 
            WHEN td.first_commit_ts < td.first_view_ts THEN 'before'
            ELSE 'after'
        END as time,
        AVG((julianday(td.first_commit_ts) - julianday(td.deadlines)) * 24) as avg_diff
    FROM (
        SELECT t.uid, t.labname, t.first_commit_ts, t.first_view_ts, DATETIME(d.deadlines, 'unixepoch') AS deadlines
        FROM test AS t LEFT JOIN deadlines AS d ON t.labname = d.labs
        WHERE t.uid LIKE 'user_%' and t.labname <> "project1"
    ) AS td
    GROUP BY 
        CASE 
            WHEN td.first_commit_ts < td.first_view_ts THEN 'before'
            ELSE 'after'
        END
    ORDER BY time DESC
    """,
    con=conn
)

In [20]:
test_results

,time,avg_diff
0,before,-61.156438
1,after,-103.953310


In [22]:
control_results = pd.io.sql.read_sql(
    """
    SELECT 
        CASE 
            WHEN cd.first_commit_ts < cd.first_view_ts THEN 'before'
            ELSE 'after'
        END as time,
        AVG((julianday(cd.first_commit_ts) - julianday(cd.deadlines)) * 24) as avg_diff
    FROM (
        SELECT c.uid, c.labname, c.first_commit_ts, c.first_view_ts, DATETIME(d.deadlines, 'unixepoch') AS deadlines
        FROM control AS c LEFT JOIN deadlines d ON c.labname = d.labs
        WHERE c.uid LIKE 'user_%' and c.labname <> "project1"
    ) AS cd
    GROUP BY 
        CASE 
            WHEN cd.first_commit_ts < cd.first_view_ts THEN 'before'
            ELSE 'after'
        END
    ORDER BY time DESC
    """,
    con=conn
)

In [23]:
control_results

,time,avg_diff
0,before,-99.901295
1,after,-113.232196


## 4

In [24]:
conn.close()

## 5

Как можно увидеть из результатов. До первого просмотра pageviews юзер делал свой коммит в среднем за 61 час до дедлайна, после первого просмотра pageviews юзер в среднем стал коммитить за 104 часа до дедлайна, то есть в среднем на **70%**. Для контрольной группы показатель изменился не так сильно - **на 14**, что позволяет сделать вывод о том, что посещение страницы действительно влияет на поведение студента